In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/
p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
/;


p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3


In [3]:
sub subp { my ($a, $b) = @_; return [ map { $a->[$_] - $b->[$_] } 0 .. $#$a ]; }
sub addp { my ($a, $b) = @_; return [ map { $a->[$_] + $b->[$_] } 0 .. $#$a ]; }
sub mulp { my ($a, $v) = @_; return [ map { $a->[$_] * $v } 0 .. $#$a ]; }
sub stringp { my ($p) = $_[0] // $_; return $p->[0].','.$p->[1] }
sub uniqp { my %h; @h{map $_->[0].','.$_->[1], @_} = (); map [ split ',' ], keys %h }
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0 && $p->[1] >= 0 && $p->[0] < $shape[0] && $p->[1] < $shape[1];
}
sub adjacent_points {
    my ($p, @shape) = @_;
    my @adj;
    return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}

In [4]:
sub nd_from_shape {
    my @shape = @_;
    return [ map undef, 0 .. $shape[0] - 1 ] if @shape == 1;
    return [ map nd_from_shape(@shape[1 .. $#shape]), 0 .. $shape[0] - 1 ];
}
sub map_robots {
    my ($shape, @robots) = @_;
    my $map = map_nd sub { 0 }, nd_from_shape(@$shape);

    foreach (@robots) {
        $map->[$_->[0][0]][$_->[0][1]]++;
    }
    $map = map_nd sub { $_ > 0 ? $_ : '.' }, $map;
    return $map;
}

In [5]:
# 10 + 10 * -int -1/10
sub wrap_point {
    my ($p, @shape) = @_;
    my ($y,$x) = @$p;
    $y += $shape[0] + $shape[0] * -int($y/$shape[0]) if $y < 0;
    $y %= $shape[0];
    
    $x += $shape[1] + $shape[1] * -int($x/$shape[1]) if $x < 0;
    $x %= $shape[1];

    return [ $y, $x ];
}
# say Dumper wrap_point([-71,7], 7,7);

In [6]:
sub process_day14_part1 {
    my ($input, $gridsize, $steps) = @_;
    my @shape = @$gridsize;
    my @half = map int($_/2), @shape;
    my @robots =
        map { die "assertion failed:", Dumper $_ unless in_bounds($_->[0], @shape); $_ }
        map { die unless /p=(\d+),(\d+) v=(-?\d+),(-?\d+)/; [ [$2, $1], [$4, $3] ] }
        grep $_,
        split "\n", $input;
    @robots =
        map { die "assertion failed:", Dumper $_ unless in_bounds($_->[0], @shape); $_ }
        map { [ wrap_point(addp($_->[0], mulp($_->[1], $steps)), @shape), $_->[1] ] }
        @robots;
    my $map = map_robots(\@shape, @robots);
    say "debug: map:\n", string_2d_map_array $map;
    my $topleft = slice_nd [[0, $half[0] - 1], [0,$half[1] - 1]], $map;
    my $topright = slice_nd [[0, $half[0] - 1], [$half[1] + 1]], $map;
    my $bottomleft = slice_nd [[$half[0] + 1], [0,$half[1] - 1]], $map;
    my $bottomright = slice_nd [[$half[0] + 1], [$half[1] + 1]], $map;
    say "debug: topleft:\n", string_2d_map_array $topleft;
    say "debug: topright:\n", string_2d_map_array $topright;
    say "debug: bottomleft:\n", string_2d_map_array $bottomleft;
    say "debug: bottomright:\n", string_2d_map_array $bottomright;

    product
        map { sum map { $_ eq '.' ? 0 : $_ } flatten_nd $_ }
        $topleft, $topright, $bottomleft, $bottomright;
}
timer sub {
    # say Dumper process_day14_part1($practice_input, [7,11], 100);
    confirm process_day14_part1($practice_input, [7,11], 100), 12;
}

debug: map:
......2..1.
...........
1..........
.11........
.....1.....
...12......
.1....1....
debug: topleft:
.....
.....
1....
debug: topright:
2..1.
.....
.....
debug: bottomleft:
.....
...12
.1...
debug: bottomright:
.....
.....
1....
confirmed: 12
[time] 0 seconds


0

In [7]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/14/input');
    # say $input;
    $res = process_day14_part1($input, [103, 101], 100);
    say Dumper $res;
    confirm $res, 209409792;
};

debug: map:
................................1.........1.................1.....1.....1............................
..........1..........................1........................1......................1...............
...1...................1.......................1............11....................................1..
...................1.....1......................................................11...................
........................1....................................................1.....................1.
.......................................1..................................1.1........................
.......1...................................1...........1.....................1.......................
....................................................................................2................
......1......................1.........................1.................................1...........
.........1.....1.............1...........1.............1..............

0

In [35]:
say post_answer('2024/day/14/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 14 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

## checking looping conditions
The entire system loops every `10403` iterations. It is pointless to check beyond that.

In [56]:
sub appendfile { local $/; my $file = IO::File->new($_[0], 'a'); $file->print($_[1]) }
sub any (&@) { my ($fun, @args) = @_; foreach (@args) { return 1 if $fun->() } return 0 }
sub process_day14_part2 {
    my ($input, $gridsize, $max_steps) = @_;
    my @shape = @$gridsize;
    my $steps = 1;
    my @robots =
        map { die "assertion failed:", Dumper $_ unless in_bounds($_->[0], @shape); $_ }
        map { die unless /p=(\d+),(\d+) v=(-?\d+),(-?\d+)/; [ [$2, $1], [$4, $3] ] }
        grep $_,
        split "\n", $input;

    # @robots =
    #     # map { die "assertion failed:", Dumper $_ unless in_bounds($_->[0], @shape); $_ }
    #     map {
    #         my $p = $_->[0];
    #         my $v = $_->[1];
    #         my ($y, $x) = @$v;
    #         (grep defined, map { ($_*$y)%$shape[0] == 0 ? $_ : undef } 1 .. $shape[0])[0];
    #         # [ wrap_point(addp($_->[0], mulp($_->[1], $steps)), @shape), $_->[1] ]
    #     }
    #     @robots;

    foreach my $step (1 .. $max_steps) {
        @robots =
            map { die "assertion failed:", Dumper $_ unless in_bounds($_->[0], @shape); $_ }
            map { [ wrap_point(addp($_->[0], mulp($_->[1], $steps)), @shape), $_->[1] ] }
            @robots;
        my $map = map_robots(\@shape, @robots);
        my $map_transposed = transpose_2d $map;
        # say string_2d_map_array $map_transposed;
        my @tracey = map { sum map { $_ eq '.' ? 0 : $_ } @{$map->[$_]} } 0 .. $shape[0]-1;
        my @tracex = map { sum map { $_ eq '.' ? 0 : $_ } @{$map_transposed->[$_]} } 0 .. $shape[1]-1;
        # say Dumper \@tracey;
        # say Dumper \@tracex;
        # say "trace_row:", Dumper \@tracex if any { $_ > 20 } @tracex;
        # appendfile('.exp/day14_tree_traces', "\n\nstep ".($step*$steps).":\n" . join(',', @tracey) . "\n" . join(',', @tracex));
        # appendfile('.exp/day14_tree_traces', "\n\nstep ".($step*$steps).":\n" . (string_2d_map_array ($map) =~ s/\./ /gr));
        if ((any { $_ > 30 } @tracex) or (any { $_ > 30 } @tracey)) {
            appendfile('.exp/day14_tree_traces_interest', "\n\nstep ".($step*$steps).":\n" . join(',', @tracey) . "\n" . join(',', @tracex) . (string_2d_map_array ($map) =~ s/\./ /gr));
            # appendfile('.exp/day14_tree_traces_interest', "\n\nstep ".($step*$steps).":\n" . (string_2d_map_array ($map) =~ s/\./ /gr));
        }
    }
    # my $map = map_robots(\@shape, @robots);
    # say "debug: map:\n", string_2d_map_array $map;
}
timer sub {
    my $input = get_challenge('2024/day/14/input');
    say Dumper process_day14_part2($input, [103, 101], 10000);
    # confirm process_day14_part1($practice_input, [7,11], 100), 12;
}

$VAR1 = '';

[time] 207 seconds


207

Warning: Subroutine appendfile redefined at reply input line 1.

Subroutine any redefined at reply input line 2.

Subroutine process_day14_part2 redefined at reply input line 3.


In [ ]:
# 8006